In [2]:
!pip install pampy

You should consider upgrading via the '/Users/user/.pyenv/versions/3.9.4/bin/python3.9 -m pip install --upgrade pip' command.


In [23]:
!pip install python-telegram-bot --upgrade

     |████████████████████████████████| 497 kB 538 kB/s eta 0:00:01
     |████████████████████████████████| 58 kB 9.7 MB/s  eta 0:00:01
     |████████████████████████████████| 339 kB 7.6 MB/s eta 0:00:01
You should consider upgrading via the '/Users/user/.pyenv/versions/3.9.4/bin/python3.9 -m pip install --upgrade pip' command.


In [63]:
!pip install pynori

     |████████████████████████████████| 42.3 MB 29.9 MB/s eta 0:00:01
     |████████████████████████████████| 983 kB 12.6 MB/s eta 0:00:01
You should consider upgrading via the '/Users/user/.pyenv/versions/3.9.4/bin/python3.9 -m pip install --upgrade pip' command.


In [70]:
!pip3 install konlpy
!pip3 install customized_konlpy

     |████████████████████████████████| 19.4 MB 9.3 MB/s eta 0:00:01
     |████████████████████████████████| 381 kB 10.7 MB/s eta 0:00:01
You should consider upgrading via the '/Users/user/.pyenv/versions/3.9.4/bin/python3.9 -m pip install --upgrade pip' command.
     |████████████████████████████████| 881 kB 317 kB/s eta 0:00:01
You should consider upgrading via the '/Users/user/.pyenv/versions/3.9.4/bin/python3.9 -m pip install --upgrade pip' command.


In [73]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from functools import reduce
import operator
import json
from pampy import match
import re
from requests_html import HTMLSession
session = HTMLSession()
import telegram

from pynori.korean_analyzer import KoreanAnalyzer
nori = KoreanAnalyzer(
           decompound_mode='NONE', # DISCARD or MIXED or NONE
           infl_decompound_mode='NONE', # DISCARD or MIXED or NONE
           discard_punctuation=True,
           output_unknown_unigrams=False,
           pos_filter=True, stop_tags=['JKS', 'JKB', 'VV', 'EF', 'EP'],
           synonym_filter=False, mode_synonym='NORM', # NORM or EXTENSION
       ) 

print(nori.do_analysis("아빠가 방에 들어가신다."))

from ckonlpy.tag import Twitter
twitter = Twitter()
twitter.pos("아빠가 방에 들어가신다.")

{'termAtt': ['아빠', '방'], 'offsetAtt': [(0, 2), (4, 5)], 'posLengthAtt': [1, 1], 'posTypeAtt': ['MORP', 'MORP'], 'posTagAtt': ['NNG', 'NNG'], 'dictTypeAtt': ['KN', 'KN']}


[('아빠', 'Noun'),
 ('가', 'Josa'),
 ('방', 'Noun'),
 ('에', 'Josa'),
 ('들어', 'Verb'),
 ('가신', 'Noun'),
 ('다', 'Josa'),
 ('.', 'Punctuation')]

In [22]:
from IPython.core.display import display, HTML
display(HTML('''
<!DOCTYPE html>
<html>
<head>
  <link href="https://fonts.googleapis.com/css?family=Roboto:100,300,400,500,700,900|Material+Icons" rel="stylesheet">
  <link href="https://cdn.jsdelivr.net/npm/vuetify/dist/vuetify.min.css" rel="stylesheet">
  <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no, minimal-ui">
</head>
<body>
  <div id="app">
    <v-app>
      <v-content>
          <v-layout row>
            <v-flex xs12 sm2 offset-sm3>
              <v-card>
                <v-toolbar color="pink" dark>
                  <v-toolbar-title>Inbox</v-toolbar-title>
                </v-toolbar>

                <v-list>
                  <template v-for="(item, index) in items">
                    <v-list-tile
                      :key="item.title"
                      @click="toggle(index)"
                    >
                      <v-list-tile-content>
                        <v-list-tile-title>{{ item.title }}</v-list-tile-title>
                      </v-list-tile-content>
                    </v-list-tile>
                  </template>
                </v-list>
              </v-card>
            </v-flex>
          </v-layout>
      </v-content>
    </v-app>
  </div>

  <script src="https://cdn.jsdelivr.net/npm/vue/dist/vue.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/vuetify/dist/vuetify.js"></script>
  <script>
    new Vue({ 
        el: '#app',
        data: {
            selected: [2],
            items: [
              {
                action: '15 min',
                headline: 'Brunch this weekend?',
                title: 'Ali Connors',
                subtitle: "I'll be in your neighborhood doing errands this weekend. Do you want to hang out?"
              },
              {
                action: '2 hr',
                headline: 'Summer BBQ',
                title: 'me, Scrott, Jennifer',
                subtitle: "Wish I could come, but I'm out of town this weekend."
              },
              {
                action: '6 hr',
                headline: 'Oui oui',
                title: 'Sandra Adams',
                subtitle: 'Do you have Paris recommendations? Have you ever been?'
              },
              {
                action: '12 hr',
                headline: 'Birthday gift',
                title: 'Trevor Hansen',
                subtitle: 'Have any ideas about what we should get Heidi for her birthday?'
              },
              {
                action: '12 hr',
                headline: 'Birthday gift',
                title: 'Trevor Hansen',
                subtitle: 'Have any ideas about what we should get Heidi for her birthday?'
              },
              {
                action: '12 hr',
                headline: 'Birthday gift',
                title: 'Trevor Hansen',
                subtitle: 'Have any ideas about what we should get Heidi for her birthday?'
              },
              {
                action: '12 hr',
                headline: 'Birthday gift',
                title: 'Trevor Hansen',
                subtitle: 'Have any ideas about what we should get Heidi for her birthday?'
              },
              {
                action: '18hr',
                headline: 'Recipe to try',
                title: 'Britta Holt',
                subtitle: 'We should eat this: Grate, Squash, Corn, and tomatillo Tacos.'
              }
            ]        
        },
        methods: {
         toggle (index) {
            const i = this.selected.indexOf(index)

            if (i > -1) {
              this.selected.splice(i, 1)
            } else {
              this.selected.push(index)
            }
          }        
        }
    })
  </script>
</body>
</html>
'''))

In [58]:
def get_media_list():
    r = session.get('https://m.news.naver.com/newspaper/home.nhn')
    return [{'id': li.attrs['id'], 'title': li.find('img', first=True).attrs['alt']} for li in r.html.find('ul.offc_lst._headline_list > li')]

media_dict = {media['id']: media for media in get_media_list()}

def get_article_list(media, ymd):
    url = 'https://media.naver.com/press/{}/newspaper?date={}'
    r = session.get(url.format(media, ymd))
    article_list = [{'title': article.find('strong', first=True).text, 'link': article.find('a', first=True).attrs['href']} for article in r.html.find('ul.newspaper_article_lst > li')]
    return article_list

def get_summary(article_link):
    m = match(article_link, re.compile('https://.+/.+/.+/([0-9]+)/([0-9]+)'), lambda m, a: (m, a))
    r = session.get('https://tts.news.naver.com/article/{}/{}/summary'.format(*m))
    rj = r.json()
    if 'summary' in rj:
        return [summary for summary in rj['summary'].split('<br/>') if summary]
    return None

def get_content(article_link):
  r = session.get(article_link)
  return r.html.find('div#newsct_article')[0]

def get_telegram_message(title, day, index, article):
    message = [f"[{title}({day})-{index}]({article['link']})"]
    for summary in get_summary(article['link']):
        message.append(f"- {summary}")
    return '\n'.join(message)


def send_telegram(media, day):
    chat_token = "5349929547:AAHd2FSvbKsVNFdN-fK8uXtzqVts9jYrodU"
    chat = telegram.Bot(token = chat_token)
    for i, article in enumerate(get_article_list(media, day)):
        message = get_telegram_message(media_dict[media]['title'], day, i, article)
        chat.sendMessage(chat_id, parse_mode='Markdown', text=message)    

In [46]:
media_dict

{'009': {'id': '009', 'title': '매일경제'},
 '015': {'id': '015', 'title': '한국경제'},
 '011': {'id': '011', 'title': '서울경제'},
 '353': {'id': '353', 'title': '중앙SUNDAY'},
 '088': {'id': '088', 'title': '매일신문'},
 '020': {'id': '020', 'title': '동아일보'},
 '028': {'id': '028', 'title': '한겨레'},
 '005': {'id': '005', 'title': '국민일보'},
 '032': {'id': '032', 'title': '경향신문'},
 '022': {'id': '022', 'title': '세계일보'},
 '023': {'id': '023', 'title': '조선일보'},
 '469': {'id': '469', 'title': '한국일보'},
 '021': {'id': '021', 'title': '문화일보'},
 '082': {'id': '082', 'title': '부산일보'},
 '016': {'id': '016', 'title': '헤럴드경제'},
 '029': {'id': '029', 'title': '디지털타임스'},
 '014': {'id': '014', 'title': '파이낸셜뉴스'},
 '008': {'id': '008', 'title': '머니투데이'},
 '081': {'id': '081', 'title': '서울신문'},
 '030': {'id': '030', 'title': '전자신문'},
 '640': {'id': '640', 'title': '코리아중앙데일리'},
 '087': {'id': '087', 'title': '강원일보'},
 '277': {'id': '277', 'title': '아시아경제'},
 '044': {'id': '044', 'title': '코리아헤럴드'},
 '018': {'id': '018', 't

In [105]:
send_telegram('030', '20220422')

In [59]:
article_list = get_article_list('009', '20220423')

In [83]:
get_content(article_list[0]['link']).text

'파월 "조금더 빨리 움직여야"\n5월 금리 0.5%P 인상 예고\n\n원화값 장중 1240원 넘고\n코스피 하락해 2700선 턱걸이\n\n◆ 美연준 빅스텝 공식화 ◆\n\n\n[AFP = 연합뉴스] 제롬 파월 미국 연방준비제도 의장이 5월 3~4일 연방공개시장위원회(FOMC) 회의에서 기준금리를 한 번에 0.5%포인트 올리는 \'빅스텝\'을 기정사실화했다. 지난 3월 빅스텝 가능성을 언급한 데 이어 오는 5월 FOMC 회의에서 \'빅스텝\'을 검토할 것이라고 공식화한 것이다.\n\n파월 의장이 금리 인상기 초반에 금리를 크게 올리는 \'선취 방식(front-end loading)\'도 언급함에 따라 5~7월 연준이 연달아 세 차례의 \'빅스텝\'을 밟을 가능성도 제기되고 있다. 파월 의장의 빅스텝 발언에 외환시장은 크게 요동쳤다. 22일 달러당 원화값은 장중 한때 1245.4원까지 하락해 연중 최저치를 경신했다. 하지만 일본과 미국의 재무장관이 최근 엔화 약세와 관련해 논의했다는 소식이 전해지며 달러당 원화값은 전날 종가(1239.0원)보다 0.1원 하락한 1239.1원으로 마감했다.\n\n월스트리트저널(WSJ) 등에 따르면 21일(현지시간) 미 워싱턴DC에서 열린 국제통화기금(IMF) 주최 패널 토론에 참석한 파월 의장은 금리 인상과 관련해 "연준이 좀 더 빨리 움직이는 게 적절하다"면서 "기준금리 0.5%포인트 인상이 5월 회의 테이블에서 검토될 것"이라고 밝혔다.\n\n파월 의장이 강경 매파(긴축적 통화정책 선호)로 기운 것은 물가가 좀처럼 꺾이지 않고 있기 때문이다. 지난 3월 미국 소비자물가상승률은 8.5%로 1981년 12월 이후 약 40년 만에 최대치를 기록했다. 이날 파월 의장은 1980년대 초 두 자릿수에 달하던 물가상승률을 잡아 \'인플레이션 파이터\'로 불려온 폴 볼커 전 연준 의장까지 언급하며 강력한 대응 의지를 드러냈다.\n\n이날 뉴욕 증시는 파월 의장의 발언에 일제히 하락세로 돌아섰으며 미 10년물 국채 금리는 한때 코로나19 팬데

In [84]:
nori.do_analysis(get_content(article_list[0]['link']).text)['termAtt']

['파월',
 '조금',
 '더',
 '빨리',
 '5',
 '월',
 '금리',
 '0',
 '5',
 'p',
 '인상',
 '예고',
 '원화',
 '값',
 '장중',
 '1240',
 '원',
 '고',
 '코스피',
 '하락',
 '해',
 '2700',
 '선',
 '턱걸이',
 '美',
 '연준',
 '빅',
 '스텝',
 '공식',
 '화',
 'afp',
 '연합뉴스',
 '제롬',
 '파월',
 '미국',
 '연방',
 '준비',
 '제도',
 '의장',
 '5',
 '월',
 '3',
 '4',
 '일',
 '연방',
 '공개',
 '시장',
 '위원회',
 'fomc',
 '회의',
 '기준',
 '금리',
 '를',
 '한',
 '번',
 '0',
 '5',
 '포인트',
 '는',
 '빅',
 '스텝',
 '을',
 '기정사실',
 '화',
 '3',
 '월',
 '빅스',
 '텝',
 '가능',
 '성',
 '을',
 '언급',
 '한',
 '데',
 '어',
 '는',
 '5',
 '월',
 'fomc',
 '회의',
 '빅',
 '스텝',
 '을',
 '검토',
 '할',
 '것',
 '이',
 '라고',
 '공식',
 '화',
 '한',
 '것',
 '이',
 '파월',
 '의장',
 '금리',
 '인상기',
 '초반',
 '금리',
 '를',
 '크',
 '게',
 '는',
 '선취',
 '방식',
 'front',
 'end',
 'loading',
 '도',
 '언급',
 '함',
 '5',
 '7',
 '월',
 '연준',
 '이',
 '아',
 '세',
 '차례',
 '의',
 '빅',
 '스텝',
 '을',
 '을',
 '가능',
 '성',
 '도',
 '제기',
 '되',
 '고',
 '있',
 '파월',
 '의장',
 '의',
 '빅',
 '스텝',
 '발언',
 '외환',
 '시장',
 '은',
 '크',
 '게',
 '22',
 '일',
 '달러',
 '당',
 '원화',
 '값',
 '은',
 '장중',
 

In [101]:
twitter.add_dictionary('인상기', 'Noun')

In [103]:
twitter.pos(get_content(article_list[0]['link']).text)

[('파월', 'Noun'),
 ('"', 'Punctuation'),
 ('조금', 'Noun'),
 ('더', 'Noun'),
 ('빨리', 'Adverb'),
 ('움직', 'Noun'),
 ('여야', 'Noun'),
 ('"', 'Punctuation'),
 ('5월', 'Number'),
 ('금리', 'Noun'),
 ('0.5%', 'Number'),
 ('P', 'Alpha'),
 ('인상', 'Noun'),
 ('예고', 'Noun'),
 ('원화', 'Noun'),
 ('값', 'Noun'),
 ('장중', 'Noun'),
 ('1240원', 'Number'),
 ('넘고', 'Verb'),
 ('코스피', 'Noun'),
 ('하락', 'Noun'),
 ('해', 'Josa'),
 ('2700', 'Number'),
 ('선', 'Noun'),
 ('턱걸이', 'Noun'),
 ('◆', 'Foreign'),
 ('美연준', 'Noun'),
 ('빅스텝', 'Noun'),
 ('공식화', 'Noun'),
 ('◆', 'Foreign'),
 ('[', 'Punctuation'),
 ('AFP', 'Alpha'),
 ('=', 'Punctuation'),
 ('연합뉴스', 'Noun'),
 (']', 'Punctuation'),
 ('제롬', 'Noun'),
 ('파월', 'Noun'),
 ('미국', 'Noun'),
 ('연방준비제도', 'Noun'),
 ('의장', 'Noun'),
 ('이', 'Josa'),
 ('5월', 'Number'),
 ('3~4일', 'Number'),
 ('연방공개시장위원회', 'Noun'),
 ('(', 'Punctuation'),
 ('FOMC', 'Alpha'),
 (')', 'Punctuation'),
 ('회의', 'Noun'),
 ('에서', 'Josa'),
 ('기준금리', 'Noun'),
 ('를', 'Josa'),
 ('한', 'Verb'),
 ('번', 'Noun'),
 ('에', 'Josa'

In [42]:
print(get_message(article_list[0]))

[여야 ‘검수완박’ 합의에 반발, 검찰 지휘부 초유의 총사퇴](https://n.news.naver.com/article/newspaper/353/0000042086?date=20220423)
 - 여야가 22일 박병석 국회의장이 제시한 '검수완박' 관련 중재안에 전격 합의했다.
 - 박 의장은 "양당 입장의 간극이 워낙 커서 선뜻 동의하기 어려웠을 텐데 뜻을 함께해 준 데 대해 감사드린다"며 "검찰개혁과 관련한 소모적 논쟁은 이젠 중단하고 앞으로는 민생과 국민을 위한 국회, 다시 신뢰받는 국회가 되길 희망한다"고 밝혔다.
 - 양당이 합의한 중재안의 핵심은 검찰의 수사권과 기소권을 분리하는 것을 대원칙으로 삼아 직접 수사권은 단계적으로 폐지하되 보완 수사권은 절차와 요건을 한정해 유지한다는 내용이다.


In [6]:
article_list = get_article_list('353', '20220423')
len(article_list)

49

In [32]:
message = []
for article in article_list:
    message.append(f"[{article['title']}]({article['link']})")
    for summary in get_summary(article['link']):
        message.append(f" - {summary}")
message

['- [여야 ‘검수완박’ 합의에 반발, 검찰 지휘부 초유의 총사퇴](https://n.news.naver.com/article/newspaper/353/0000042086?date=20220423)',
 " - 여야가 22일 박병석 국회의장이 제시한 '검수완박' 관련 중재안에 전격 합의했다.",
 ' - 박 의장은 "양당 입장의 간극이 워낙 커서 선뜻 동의하기 어려웠을 텐데 뜻을 함께해 준 데 대해 감사드린다"며 "검찰개혁과 관련한 소모적 논쟁은 이젠 중단하고 앞으로는 민생과 국민을 위한 국회, 다시 신뢰받는 국회가 되길 희망한다"고 밝혔다.',
 ' - 양당이 합의한 중재안의 핵심은 검찰의 수사권과 기소권을 분리하는 것을 대원칙으로 삼아 직접 수사권은 단계적으로 폐지하되 보완 수사권은 절차와 요건을 한정해 유지한다는 내용이다.',
 '- [디지털 시대, 오프라인의 귀환](https://n.news.naver.com/article/newspaper/353/0000042080?date=20220423)',
 ' - 이승윤 디지털 문화심리학자의 책 『공간은 경험이다』의 프롤로그 첫 문장이다.',
 ' - 6개월 한시적으로 운영되는 특별한 컨셉트 스토어인데, 청담동 플래그십 스토어와의 차별점은 이 공간을 찾게 될 고객 타깃을 MZ세대에 정 조준했다는 점이다.',
 ' - 디올 관계자는 "독창적 에너지와 활력이 넘치는 성수동, 이 동네를 사랑하는 MZ세대, 그리고 디올의 헤리티지가 잘 어우러진 특별한 공간 문화를 기대해 달라"고 했다.',
 '- [25일부터 영화관·기차서 음식 섭취 가능](https://n.news.naver.com/article/newspaper/353/0000042088?date=20220423)',
 ' - 오는 25일부터는 실내 다중이용시설에서의 음식물 섭취가 허용된다.',
 ' - 코로나19 감염병 등급도 1급에서 2급으로 조정된다.',
 ' - 중앙재난안전대책본부는 영화관, 종교시설, 실내스포츠 관람장 등 실내 다중이용시설에서의 음식물 

In [13]:
print(get_summary(article_list[0]['link']))

여야가 22일 박병석 국회의장이 제시한 '검수완박' 관련 중재안에 전격 합의했다.

박 의장은 "양당 입장의 간극이 워낙 커서 선뜻 동의하기 어려웠을 텐데 뜻을 함께해 준 데 대해 감사드린다"며 "검찰개혁과 관련한 소모적 논쟁은 이젠 중단하고 앞으로는 민생과 국민을 위한 국회, 다시 신뢰받는 국회가 되길 희망한다"고 밝혔다.

양당이 합의한 중재안의 핵심은 검찰의 수사권과 기소권을 분리하는 것을 대원칙으로 삼아 직접 수사권은 단계적으로 폐지하되 보완 수사권은 절차와 요건을 한정해 유지한다는 내용이다.


In [17]:
datetime.now().strftime('%Y%m%d')

'20190504'

In [28]:
import telegram

chat_token = "5349929547:AAHd2FSvbKsVNFdN-fK8uXtzqVts9jYrodU"
chat = telegram.Bot(token = chat_token)
updates = chat.getUpdates()
for u in updates:
    print(u.message['chat']['id'])

738042920


In [43]:
chat_id = '738042920'
for article in article_list:
    chat.sendMessage(chat_id, parse_mode='Markdown', text=get_message(article))